In [1]:
# read the jobs.txt and get the download urls
import pandas as pd

jobs_file = pd.read_csv("../../../PRS_benchmark/data/software/polypred/alt_jobs.txt", sep=" ", header=None)
output_dir = "../../../PRS_benchmark/data/LD/PolyPred/"
npz_cmd_list = []
gz_cmd_list = []
check_list = []
# display(jobs_file)
# get the 11th column which is the download url
urls = jobs_file[11].tolist()
for url in urls:
    # print(url)
    npz_url = url + ".npz"
    gz_url = url + ".gz"
    npz_cmd = f"wget -P {output_dir} {npz_url}"
    gz_cmd = f"wget -P {output_dir} {gz_url}"
    npz_cmd_list.append(npz_cmd)
    gz_cmd_list.append(gz_cmd)
    check_item = url.replace("https://broad-alkesgroup-ukbb-ld.s3.amazonaws.com/UKBB_LD/", "")
    check_list.append(check_item)

# write the commands to a file
with open("download_commands_npz.sh", "w", newline='', encoding='utf-8') as f:
    f.write("#!/bin/bash\n")
    for cmd in npz_cmd_list:
        f.write(cmd)
        f.write("\n")
with open("download_commands_gz.sh", "w", newline='', encoding='utf-8') as f:
    f.write("#!/bin/bash\n")
    for cmd in gz_cmd_list:
        f.write(cmd)
        f.write("\n")

# change check_list to a dataframe
check_df = pd.DataFrame(check_list, columns=["check_item"])
# write the check_df to a file
check_df.to_csv("check_list.txt", sep="\t", index=False)

# python3 /data1/jiapl_group/lishuhua/software/PRS/Polyfun/polyfun-master/finemapper.py --chr 1 --start 1 --end 3000001 --out /data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/2_finemapping_jobs/alt.chr1.1_3000001.gz --ld /data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/reference/UKBB_LD/chr1_1_3000001.npz --method susie --sumstats /data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/1_parquet/alt_snpvar.txt --n 336922 --memory 1 --threads 2 --max-num-causal 10

# python3 /data1/jiapl_group/lishuhua/software/PRS/Polyfun/polyfun-master/finemapper.py --chr 1 --start 1 --end 3000001 --out /data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/2_finemapping_jobs/alt.chr1.1_3000001.gz --ld https://broad-alkesgroup-ukbb-ld.s3.amazonaws.com/UKBB_LD/chr1_1_3000001 --method susie --sumstats /data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/1_parquet/alt_snpvar.txt --n 336922 --memory 1 --threads 2 --max-num-causal 10




In [ ]:
# change all url in _jobs.txt to local path
import os
import pandas as pd

jobs_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/2_finemapping_jobs/"
ref_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/reference/UKBB_LD/"

trait_dict = {
    'p48': 'waist',
    'p50': 'height',
    'p102': 'pulse',
    'p4079': 'dbp',
    'p4080': 'sbp',
    'p20116': 'smoke',
    'p20117': 'drink',
    'p21001': 'bmi',
    'p30000': 'wbc',
    'p30010': 'rbc',
    'p30020':'hb',
    'p30080': 'plt',
    'p30120': 'lymph',
    'p30130': 'mono',
    'p30140': 'neut',
    'p30150': 'eos',
    'p30620': 'alt',
    'p30650': 'ast',
    'p30670': 'bun',
    'p30690': 'cholesterol',
    'p30700': 'creatinine',
    'p30730': 'ggt',
    'p30740': 'glucose',
    'p30760': 'hdl',
    'p30780': 'ldl',
    'p30870': 'triglycerides',
    'p30880': 'ua'
}

for trait, name in trait_dict.items():
    create_dir = os.path.join(jobs_dir, name)
    if not os.path.exists(create_dir):
        os.makedirs(create_dir)
    # check if there are any files start with name in the jobs_dir
    for file in os.listdir(jobs_dir):
        if file.startswith(name) and (file.endswith(".txt") or file.endswith(".log")):
            old_path = os.path.join(jobs_dir, file)
            new_path = os.path.join(create_dir, file)
            os.rename(old_path, new_path)
            print(f"Moved {file} to {create_dir}")
    # for each file in the new directory, replace the url with local path
    for job_file in os.listdir(create_dir):
        if job_file.endswith("_jobs.txt"):
            output_name = job_file.replace("_jobs.txt", "_jobs_updated.txt")
            file_path = os.path.join(create_dir, job_file)
            df = pd.read_csv(file_path, sep=" ", header=None)
            # replace the 11th column with local path
            df[11] = df[11].str.replace("https://broad-alkesgroup-ukbb-ld.s3.amazonaws.com/UKBB_LD/", ref_dir)
            # df[11] = df[11].str.replace("/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/train/EUR/2_finemapping_jobs/reference/UKBB_LD/", ref_dir)
            output_prefix = os.path.join(create_dir, output_name)
            df.to_csv(output_prefix, sep=" ", index=False, header=False)
            print(f"Updated {file_path} and saved to {output_prefix}")

In [ ]:
# check if all .npz and .gz files are in the reference directory
import os
import pandas as pd
ref_dir = "/data1/jiapl_group/lishuhua/project/PRS_benchmark/software/polypred/reference/UKBB_LD/"
check_list = pd.read_csv("check_list.txt", sep="\t")

npz_files = [f for f in os.listdir(ref_dir) if f.endswith('.npz')]
gz_files = [f for f in os.listdir(ref_dir) if f.endswith('.gz')]

# check if all items in check_list are in npz_files and gz_files
missing_npz = [item for item in check_list['check_item'] if item + '.npz' not in npz_files]
missing_gz = [item for item in check_list['check_item'] if item + '.gz' not in gz_files]

if missing_npz:
    print(f"Missing .npz files num: {len(missing_npz)}")

if missing_gz:
    print(f"Missing .gz files num: {len(missing_gz)}")